TODO: Es posible que se puede sacar algo de aquí http://sahandsaba.com/understanding-asyncio-node-js-python-3-4.html

TODO: Y puede que de aquí: https://pymotw.com/2/multiprocessing/basics.html

TODO: Ejercicio muy sencillo: https://github.com/cmchurch/python-multiprocessing/blob/master/multiprocess-basic-pool-example.py

In [2]:
# ejemplo de zip
import threading, zipfile

class AsyncZip(threading.Thread):
    def __init__(self, infile, outfile):
        threading.Thread.__init__(self)
        self.infile = infile
        self.outfile = outfile
        
    def run(self):
        f = zipfile.ZipFile(self.outfile, 'w', zipfile.ZIP_DEFLATED)
        f.write(self.infile)
        f.close()
        print('Finished background zip of {}'.format(self.infile))

background = AsyncZip('mydata.txt', 'myarchive.zip')
background.start()
print('The main program continues to run in foreground.')

background.join() # Wait for the background task to finish
print('Main program waited until background was done.')

The main program continues to run in foreground.
Main program waited until background was done.


Exception in thread Thread-6:
Traceback (most recent call last):
  File "D:\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-cf18fed96ebd>", line 12, in run
    f.write(self.infile)
  File "D:\Anaconda3\lib\zipfile.py", line 1433, in write
    st = os.stat(filename)
FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'mydata.txt'



In [ ]:
# crawler
import http.client
import threading
import logging

logging.basicConfig(level=logging.INFO, format=’(%(threadName)-10s) %(message)s’, )

def save(html, file_absolute_path):
    logging.info("saving {} bytes to {}".format(len(html), file_absolute_path))
    with open(file_absolute_path, ’wb+’) as file:
        file.write(html)
        file.flush()

def crawl(req):
    logging.info("executing get request for parameters: {}".format(str(req)))
    connection = http.client.HTTPConnection(req["host"], req["port"])
    connection.request("GET", req["path"])
    response = connection.getresponse()
    logging.info("got {} response http code".format(response.status))
    logging.debug("headers: {}".format(str(response.headers)))
    response_content = response.read()
    logging.debug("actual response: {}".format(response_content))
    return response_content


class MyCrawler(threading.Thread):
    def __init__(self, req, file_path):
        threading.Thread.__init__(self, name="Crawler-{}".format(req["host"]), daemon=True)
        self.req = req
        self.file_path = file_path
    
    def run(self):
        global executed_crawlers
        html = crawl(self.req)
        save(html, self.file_path)
        
        

def __main__():
    continue_input = True
    threads = []
    while continue_input:
        host = input("host: ")
        port = 80 # int(input("port: "))
        path = "/" # input("path: ")
        file_path = input("output file absolute path: ")
        req = {"host": host, "port": port, "path": path}
        threads.append(MyCrawler(req, file_path))
        continue_input = input("add another? (y/N) ") == "y"

        for t in threads:
            t.start()
            
    current_thread = threading.currentThread()
    for thread in threading.enumerate():
        if thread is not current_thread:
            thread.join()

__main__()


In [ ]:
# Carrera
import threading

class Racer(threading.Thread):
    def __init__(self, name, start_signal):
        threading.Thread.__init__(self, name=name)
        self.start_signal = start_signal

    def run(self):
        self.start_signal.wait()
        print("I, {}, got to the goal!".format(self.name))

class Race:
    def __init__(self, racer_names):
        self.start_signal = threading.Event()
        self.racers = [Racer(name, self.start_signal) for name in racer_names]
        for racer in self.racers:
            racer.start()

    def start(self):
        self.start_signal.set()

def __main__():
    race = Race(["rabbit", "turtle", "cheetah", "monkey", "cow", "horse", "tiger", "lion"])
    race.start()

__main__()